Multi-Symbol Version

In [ ]:
# LIVE TRADING CODE FOR MULTI-BAR CLASSIFICATION

import sys
import os
import warnings
from pathlib import Path

# ---------------------------------------------------------------------------
# 1) SET PROJECT ROOT AND UPDATE PATH/WORKING DIRECTORY
# ---------------------------------------------------------------------------
project_root = Path.cwd().parent.parent  # Adjust if your notebook is in notebooks/time_series
sys.path.append(str(project_root))
os.chdir(str(project_root))
warnings.filterwarnings("ignore")

import warnings
warnings.filterwarnings("ignore")
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import ta
from datetime import datetime, timedelta
import time
import logging
import joblib
from features.feature_engineering import add_core_features

import sqlite3
import pandas as pd
from datetime import datetime



# Setup logging
logging.basicConfig(
    filename='models/saved_models/trading_app1.log',
    level=logging.INFO,
    format='%(asctime)s %(levelname)s:%(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

def log_and_print(message, is_error=False):
    """
    Logs and prints a message.
    If is_error=True, logs at the ERROR level; otherwise logs at INFO level.
    """
    if is_error:
        logging.error(message)
    else:
        logging.info(message)
    print(message)

# Update the login credentials and server information accordingly
#name = 66677507
#key = 'ST746$nG38'
#serv = 'ICMarketsSC-Demo'

# Global variables
symbols = [
    "US500", "US2000", "UK100", "JP225", "XAUUSD", "DE40", "US30",
    "USTEC",  "BTCUSD", "AAPL.NAS", "MSFT.NAS", "GOOG.NAS", "AMZN.NAS", "TSLA.NAS"
]

lot_sizes = {
  
    "US500": 0.1,
    "US2000": 0.1,
    "UK100": 0.1,
    "DE40": 0.1,
    "USTEC": 0.1,
    "JP225": 1.00,

}

# Add Forex and Stocks with 0.01 lot size
for symbol in symbols:
    if symbol not in lot_sizes:
        lot_sizes[symbol] = 0.01

model_paths = {
    symbol: f"models/h1_models/{symbol}_H1_best_model.pkl" for symbol in symbols
}


TIMEFRAME = mt5.TIMEFRAME_H1
N_BARS = 1000
MAGIC_NUMBER = 234003
SLEEP_TIME = 3600  # 1 hour
COMMENT_ML = "RFFV-D"

success_count = 0
fail_count = 0
retry_queue = []
# Global setting for whether to close open positions on flat (neutral) signal
CLOSE_ON_FLAT = True
N_FORWARD = 3  # Number of bars ahead to predict and save

# If you still need feature selection, you can keep this helper function:
def select_features_rf_reg(X, y, estimator, max_features=20):
    """
    Example helper function for feature selection using RandomForest.
    """
    from sklearn.feature_selection import SelectFromModel
    selector = SelectFromModel(estimator=estimator, threshold=-np.inf, max_features=max_features).fit(X, y)
    X_transformed = selector.transform(X)
    selected_features_mask = selector.get_support()
    return X_transformed, selected_features_mask

def is_us_stock(symbol):
    return symbol.endswith(".NAS") or symbol.endswith(".NYSE")

def is_us_market_open():
    """
    Check if US stock market is open based on Switzerland time (CET/CEST).
    """
    now = datetime.now()

    if now.weekday() >= 5:  # Saturday or Sunday
        return False

    market_open = now.replace(hour=15, minute=30, second=0, microsecond=0)
    market_close = now.replace(hour=22, minute=0, second=0, microsecond=0)

    return market_open <= now <= market_close




class TradingApp:

    def __init__(self, symbol, lot_size, magic_number):
        self.symbol = symbol
        self.lot_size = lot_size
        self.magic_number = magic_number
        self.model = None         # previously self.pipeline
        self.scaler = None
        self.features = None
        self.last_retrain_time = None

    def get_data(self, symbol, n, timeframe):
        """
        Fetch 'n' bars of historical data for the given symbol and timeframe.
        """
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
        rates_frame = pd.DataFrame(rates)
        rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
        rates_frame.set_index('time', inplace=True)
        return rates_frame

    def add_core_features(self, df):
        """
        Add only the core features to the DataFrame (the same ones used in model training).
        """
        df = add_core_features(df)
        return df

    def load_pipeline(self, pipeline_path):
        pipeline_loaded = joblib.load(pipeline_path)

        if isinstance(pipeline_loaded, dict):
            self.model = pipeline_loaded.get("model")
            self.scaler = pipeline_loaded.get("scaler", None)
            self.features = pipeline_loaded.get("features", None)
        else:
            self.model = pipeline_loaded
            self.scaler = None
            self.features = None

        logging.info(f"Loaded model from {pipeline_path}")
        log_and_print(f"✅ Loaded model from {pipeline_path}")





    def ml_signal_generation(self, symbol, n_bars, timeframe):
        if self.model is None:
            logging.error("❌ No model loaded.")
            return None

        df = self.get_data(symbol, n_bars, timeframe)
        df = self.add_core_features(df)
        df.fillna(method='ffill', inplace=True)

        features = self.features if self.features else [
            "sma_20", "ema_20", "kama_10", "rsi_14", "macd_diff",
            "atr_14", "obv", "rolling_std_20", "spread", "fill", "amplitude",
            "autocorr_1", "autocorr_5", "autocorr_10", "market_regime", "stationary_flag"
        ]

        X_new = df[features].dropna()

        if X_new.empty:
            logging.error(f"❌ No valid feature rows for prediction on {symbol}.")
            return None

        if self.scaler:
            X_scaled = self.scaler.transform(X_new)
        else:
            X_scaled = X_new

        preds_shifted = self.model.predict(X_scaled)
        preds = preds_shifted - 1

        if all(p == 0 for p in preds[-N_FORWARD:]):
            log_and_print(f"⚪ {symbol}: All {N_FORWARD} predictions are flat (0).")

        return preds[-N_FORWARD:]




    def orders(self, symbol, lot, is_buy=True, id_position=None, sl=None, tp=None):
        """
        Place an order (BUY or SELL) for the specified symbol and lot size.
        """
        global success_count, fail_count, retry_queue

        symbol_info = mt5.symbol_info(symbol)
        if symbol_info is None:
            log_and_print(f"Symbol {symbol} not found, can't place order.", is_error=True)
            fail_count += 1
            return "Symbol not found"

        # Make sure symbol is visible
        if not symbol_info.visible:
            if not mt5.symbol_select(symbol, True):
                log_and_print(f"Failed to select symbol {symbol}", is_error=True)
                fail_count += 1
                return "Symbol not visible or could not be selected."

        tick_info = mt5.symbol_info_tick(symbol)
        if tick_info is None:
            log_and_print(f"Could not get tick info for {symbol}.", is_error=True)
            fail_count += 1
            return "Tick info unavailable"

        # Check for valid bid/ask
        if tick_info.bid <= 0 or tick_info.ask <= 0:
            log_and_print(
                f"Zero or invalid bid/ask for {symbol}: bid={tick_info.bid}, ask={tick_info.ask}",
                is_error=True
            )
            fail_count += 1
            return "Invalid prices"

        # LOT SIZE VALIDATION
        lot = max(lot, symbol_info.volume_min)
        step = symbol_info.volume_step
        if step > 0:
            remainder = lot % step
            if remainder != 0:
                lot = lot - remainder + step
        if lot > symbol_info.volume_max:
            lot = symbol_info.volume_max

        log_and_print(
            f"Adjusted lot size to {lot} (min={symbol_info.volume_min}, "
            f"step={symbol_info.volume_step}, max={symbol_info.volume_max})"
        )

        # Force ORDER_FILLING_IOC
        filling_mode = 1  # ORDER_FILLING_IOC

        order_type = mt5.ORDER_TYPE_BUY if is_buy else mt5.ORDER_TYPE_SELL
        order_price = tick_info.ask if is_buy else tick_info.bid
        deviation = 20

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": order_type,
            "deviation": deviation,
            "magic": self.magic_number,
            "comment": COMMENT_ML,
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": filling_mode,
        }

        if sl is not None:
            request["sl"] = sl
        if tp is not None:
            request["tp"] = tp
        if id_position is not None:
            request["position"] = id_position

        log_and_print(f"Sending order request: {request}")
        result = mt5.order_send(request)

        order_type_str = "BUY" if is_buy else "SELL"

        if result is None or result.retcode != mt5.TRADE_RETCODE_DONE:
            fail_count += 1
            error_message = f"Order failed for {symbol}"
            if result:
                error_message += f", retcode={result.retcode}, comment={result.comment}"
            additional_info = (
                f"Date/Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
                f"Order Type: {order_type_str}\n"
                f"Lot Size: {lot}\n"
                f"SL: {sl if sl else 'None'}\n"
                f"TP: {tp if tp else 'None'}\n"
                f"Comment: {COMMENT_ML}\n"
                f"Request: {request}\n"
                f"Result: {result}"
            )
            log_and_print(f"Order failed details: {additional_info}", is_error=True)

            # If market closed or only closing allowed => add to retry queue
            if result is not None and result.retcode in [10018, 10044]:
                retry_queue.append((symbol, datetime.now() + timedelta(minutes=15)))
                log_and_print(f"Symbol {symbol} added to retry queue for later attempt.", is_error=False)

        else:
            success_count += 1
            success_message = f"Order successful for {symbol}, comment={result.comment}"
            additional_info = (
                f"Date/Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
                f"Order Type: {order_type_str}\n"
                f"Lot Size: {lot}\n"
                f"SL: {sl if sl else 'None'}\n"
                f"TP: {tp if tp else 'None'}\n"
                f"Comment: {COMMENT_ML}"
            )
            log_and_print(success_message)


    def get_positions_by_magic(self, symbol, magic_number):
        """
        Retrieve positions for a specific symbol and magic number.
        """
        all_positions = mt5.positions_get(symbol=symbol)
        if not all_positions:
            log_and_print("No positions found.", is_error=False)
            return []
        return [pos for pos in all_positions if pos.magic == magic_number]



    def run_strategy(self, symbol, lot, buy_signal, sell_signal):
        """
        Run the trading strategy logic based on buy/sell signals, including flat signals.
        """
        log_and_print("------------------------------------------------------------------")
        log_and_print(
            f"🟡 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}, "
            f"SYMBOL: {symbol}, BUY SIGNAL: {buy_signal}, SELL SIGNAL: {sell_signal}"
        )

        positions = self.get_positions_by_magic(symbol, self.magic_number)
        has_buy = any(pos.type == mt5.POSITION_TYPE_BUY for pos in positions)
        has_sell = any(pos.type == mt5.POSITION_TYPE_SELL for pos in positions)

        if buy_signal and not has_buy:
            if has_sell:
                log_and_print(f"🔄 {symbol}: Existing SELL position found. Attempting to close it...")
                if self.close_position(symbol, is_buy=True):
                    log_and_print(f"✅ {symbol}: Closed SELL. Placing new BUY order.")
                    self.orders(symbol, lot, is_buy=True)
                else:
                    log_and_print(f"❌ {symbol}: Failed to close SELL position.")
            else:
                log_and_print(f"🟢 {symbol}: Placing new BUY order.")
                self.orders(symbol, lot, is_buy=True)

        elif sell_signal and not has_sell:
            if has_buy:
                log_and_print(f"🔄 {symbol}: Existing BUY position found. Attempting to close it...")
                if self.close_position(symbol, is_buy=False):
                    log_and_print(f"✅ {symbol}: Closed BUY. Placing new SELL order.")
                    self.orders(symbol, lot, is_buy=False)
                else:
                    log_and_print(f"❌ {symbol}: Failed to close BUY position.")
            else:
                log_and_print(f"🔴 {symbol}: Placing new SELL order.")
                self.orders(symbol, lot, is_buy=False)

        elif not buy_signal and not sell_signal:
            if has_buy or has_sell:
                if CLOSE_ON_FLAT:
                    log_and_print(f"⚪ {symbol}: Flat signal. CLOSE_ON_FLAT is True, attempting to close open position.")
                    success = self.close_position(symbol, is_buy=has_sell)
                    if success:
                        log_and_print(f"✅ {symbol}: Flat signal - Position closed.")
                    else:
                        log_and_print(f"❌ {symbol}: Flat signal - Failed to close position.")
                else:
                    log_and_print(f"⚪ {symbol}: Flat signal but CLOSE_ON_FLAT is False. Holding current position.")
            else:
                log_and_print(f"🟤 {symbol}: Flat signal. No position open.")



    def close_position(self, symbol, is_buy):
        """
        Closes positions of the opposite type (BUY/SELL) for this app's magic number.
        """
        positions = mt5.positions_get(symbol=symbol)
        if not positions:
            log_and_print(f"No positions to close for symbol: {symbol}")
            return False

        initial_balance = mt5.account_info().balance
        closed_any = False

        for position in positions:
            if position.magic == self.magic_number and (
                (is_buy and position.type == mt5.POSITION_TYPE_SELL) or
                (not is_buy and position.type == mt5.POSITION_TYPE_BUY)
            ):
                # First try ORDER_FILLING_RETURN
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": position.volume,
                    "type": mt5.ORDER_TYPE_BUY if position.type == mt5.POSITION_TYPE_SELL else mt5.ORDER_TYPE_SELL,
                    "position": position.ticket,
                    "deviation": 20,
                    "magic": self.magic_number,
                    "comment": COMMENT_ML,
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_RETURN,  # Try RETURN first
                }
                result = mt5.order_send(close_request)

                if result is None or result.retcode != mt5.TRADE_RETCODE_DONE:
                    # Retry WITH ORDER_FILLING_IOC instead of removing filling type
                    log_and_print(f"⚠️ Filling mode error for {symbol} position {position.ticket}, retrying with ORDER_FILLING_IOC...")
                    
                    close_request["type_filling"] = mt5.ORDER_FILLING_IOC  # Force IOC
                    result_retry = mt5.order_send(close_request)

                    if result_retry is None or result_retry.retcode != mt5.TRADE_RETCODE_DONE:
                        log_and_print(f"❌ Retry failed to close {symbol} — retcode: {result_retry.retcode}, comment: {result_retry.comment}", is_error=True)
                    else:
                        log_and_print(f"✅ Retry succeeded in closing {symbol} position {position.ticket}")
                        closed_any = True
                else:
                    log_and_print(f"✅ Successfully closed position {position.ticket} for {symbol}")
                    closed_any = True

        if closed_any:
            final_balance = mt5.account_info().balance
            profit = final_balance - initial_balance
            log_and_print(f"✅ Closed positions successfully, Profit: {profit}")
            return True
        else:
            return False





    def check_and_execute_trades(self):
        """
        Convenience method to perform the entire flow:
        generate signals, run strategy, and deselect symbol.
        """
        mt5.symbol_select(self.symbol, True)
        buy, sell, _, _ = self.ml_signal_generation(self.symbol, N_BARS, TIMEFRAME)
        self.run_strategy(self.symbol, self.lot_size, buy, sell)
        mt5.symbol_select(self.symbol, False)
        log_and_print("Waiting for new signals...")

def is_market_open():
    """
    Check if the current time is within the typical Forex trading session, adjusted for CET/CEST.
    Market closes at Friday 10:00 PM CET and opens at Sunday 11:00 PM CET. 
    It is closed all day Saturday.
    """
    current_time_utc = datetime.utcnow()
    # Adjust for Central European Time (UTC+1) or Central European Summer Time (UTC+2)
    current_time_cet = (
        current_time_utc + timedelta(hours=2) 
        if time.localtime().tm_isdst 
        else current_time_utc + timedelta(hours=1)
    )

    # Friday after 10 PM CET
    if current_time_cet.weekday() == 4 and current_time_cet.hour >= 22:
        return False
    # Sunday before 11 PM CET
    elif current_time_cet.weekday() == 6 and current_time_cet.hour < 23:
        return False
    # All day Saturday
    elif current_time_cet.weekday() == 5:
        return False
    return True




def save_signal_to_db(symbol, prediction, timestamp=None):
    conn = sqlite3.connect('live_signals.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS signals (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            symbol TEXT,
            prediction INTEGER,
            timestamp TEXT,
            UNIQUE(symbol, prediction, timestamp)
        )
    ''')
    if timestamp is None:
        timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    try:
        c.execute(
            'INSERT OR IGNORE INTO signals (symbol, prediction, timestamp) VALUES (?, ?, ?)',
            (symbol, prediction, timestamp)
        )
    except Exception as e:
        print(f"Failed to save to db: {e}")
    conn.commit()
    conn.close()



if __name__ == "__main__":
    try:
        if not mt5.initialize():
            log_and_print("Failed to initialize MetaTrader 5", is_error=True)
            exit()

        # 1) Create a TradingApp per symbol
        apps = {}
        for symbol in symbols:
            app = TradingApp(symbol=symbol, lot_size=lot_sizes[symbol], magic_number=MAGIC_NUMBER)
            app.load_pipeline(model_paths[symbol])
            apps[symbol] = app

        # 2) Initialize tracking variables
        success_count = 0
        fail_count = 0
        retry_queue = []

        while True:
            log_and_print("Checking market status...")
            if is_market_open():
                log_and_print("Market is open. Executing trades...")

                # Handle retry queue
                now = datetime.now()
                retry_symbols_ready = [item for item in retry_queue if item[1] <= now]
                retry_queue = [item for item in retry_queue if item[1] > now]

                symbols_to_process = set(apps.keys())

                # Add retry symbols first
                symbols_ready_to_retry = [item[0] for item in retry_symbols_ready]
                if symbols_ready_to_retry:
                    log_and_print(f"🔁 Retrying symbols: {symbols_ready_to_retry}")
                    symbols_to_process = set(symbols_ready_to_retry) | symbols_to_process

                for symbol in symbols_to_process:
                    app = apps[symbol]
                    try:
                        if is_us_stock(symbol) and not is_us_market_open():
                            log_and_print(f"🟠 Skipping {symbol}: US market not open yet.")
                            continue

                        log_and_print(f"🔵 Checking Symbol: {symbol}")

                        # --- Get N-step-ahead predictions ---
                        multi_preds = app.ml_signal_generation(
                            symbol=app.symbol,
                            n_bars=N_BARS,
                            timeframe=TIMEFRAME
                        )
                        if multi_preds is None:
                            continue

                        # --- Get last bar time for timestamping ---
                        df_data = app.get_data(app.symbol, N_BARS, TIMEFRAME)
                        last_bar_time = df_data.index[-1]

                        # --- Save all N_FORWARD predictions ---
                        for i, pred in enumerate(multi_preds):
                            future_time = (last_bar_time + pd.Timedelta(hours=i+1)).strftime('%Y-%m-%d %H:%M:%S')
                            save_signal_to_db(app.symbol, int(pred), timestamp=future_time)

                        # --- Use first prediction for live trading ---
                        buy_signal = (multi_preds[0] == 1)
                        sell_signal = (multi_preds[0] == -1)
                        app.run_strategy(app.symbol, app.lot_size, buy_signal, sell_signal)

                    except Exception as e:
                        log_and_print(f"⚠️ Error processing {symbol}: {e}", is_error=True)


                # After the trading round, log results
                log_and_print(f"✅ Successful Orders: {success_count}")
                log_and_print(f"❌ Failed Orders: {fail_count}")
                log_and_print(f"⏳ Retry Queue: {[item[0] for item in retry_queue]}")

            else:
                log_and_print("Market is closed. Waiting...")

            time.sleep(SLEEP_TIME)

    except KeyboardInterrupt:
        log_and_print("Shutdown signal received.")

    except Exception as e:
        error_message = f"An error occurred: {e}"
        log_and_print(error_message, is_error=True)

    finally:
        mt5.shutdown()
        log_and_print("MetaTrader 5 shutdown completed.")






Loaded pipeline from models/h1_models/US500_H1_best_model.pkl
Loaded pipeline from models/h1_models/US2000_H1_best_model.pkl
Loaded pipeline from models/h1_models/UK100_H1_best_model.pkl
Loaded pipeline from models/h1_models/JP225_H1_best_model.pkl
Loaded pipeline from models/h1_models/XAUUSD_H1_best_model.pkl
Loaded pipeline from models/h1_models/DE40_H1_best_model.pkl
Loaded pipeline from models/h1_models/US30_H1_best_model.pkl
Loaded pipeline from models/h1_models/USTEC_H1_best_model.pkl
Loaded pipeline from models/h1_models/BTCUSD_H1_best_model.pkl
Loaded pipeline from models/h1_models/AAPL.NAS_H1_best_model.pkl
Loaded pipeline from models/h1_models/MSFT.NAS_H1_best_model.pkl
Loaded pipeline from models/h1_models/GOOG.NAS_H1_best_model.pkl
Loaded pipeline from models/h1_models/AMZN.NAS_H1_best_model.pkl
Loaded pipeline from models/h1_models/TSLA.NAS_H1_best_model.pkl
Checking market status...
Market is open. Executing trades...
🟠 Skipping AAPL.NAS: US market not open yet.
🔵 Checkin